# Import

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import subprocess
import RNA
from Bio import Entrez, SeqIO
import pandas as pd
from base64 import b64encode
import json
from urllib import request, parse
import requests

%matplotlib inline

# primer generation and alignment

In [2]:
# ---------- Настройки Entrez ----------
Entrez.email = 'your_email@example.com'

# ---------- Шаг 0: Загрузка последовательности из NCBI ----------
def fetch_gene_sequence(gene_name, database='nucleotide'):
    try:
        handle = Entrez.esearch(db=database, term=gene_name, retmax=1)
        record = Entrez.read(handle)
        handle.close()

        if not record['IdList']:
            raise ValueError(f"Ген '{gene_name}' не найден в базе данных {database}.")

        gene_id = record['IdList'][0]

        handle = Entrez.efetch(db=database, id=gene_id, rettype='fasta', retmode='text')
        record = SeqIO.read(handle, 'fasta')
        handle.close()

        print(f"Последовательность гена '{gene_name}' успешно загружена.")
        return str(record.seq)
    except Exception as e:
        raise RuntimeError(f'Ошибка при загрузке последовательности: {e}')

# ---------- Шаг 1: Генерация праймеров с помощью Primer3 ----------
def run_primer3(sequence):
    if len(sequence) < 50:
        raise ValueError('Длина последовательности слишком мала для генерации праймеров!')
    input_data = f"""
SEQUENCE_ID=example
SEQUENCE_TEMPLATE={sequence}
PRIMER_OPT_SIZE=20
PRIMER_MIN_SIZE=18
PRIMER_MAX_SIZE=25
PRIMER_PRODUCT_SIZE_RANGE=100-250
PRIMER_MIN_TM=55.0
PRIMER_OPT_TM=65.0
PRIMER_MAX_TM=70.0
PRIMER_MIN_GC=40
PRIMER_MAX_GC=60
PRIMER_NUM_RETURN=5
PRIMER_MAX_HAIRPIN_TH=20.0
PRIMER_MAX_SELF_ANY_TH=20.0
PRIMER_MAX_SELF_END_TH=20.0
SEQUENCE_INCLUDED_REGION=100,500
=
"""
    with open('primer3_input.txt', 'w') as f:
        f.write(input_data.strip())
    
    subprocess.run(['primer3_core', 'primer3_input.txt', '-output', 'primer3_output.txt'])
    with open('primer3_output.txt', 'r') as f:
        return f.read()

def extract_primer(primer_output, pair_index, side):
    key = f'PRIMER_{side}_{pair_index}_SEQUENCE='
    for line in primer_output.split('\n'):
        if line.startswith(key):
            return line.split('=')[-1]
    return None

def extract_primer_tm(primer_output, pair_index, side):
    key = f'PRIMER_{side}_{pair_index}_TM='
    for line in primer_output.split('\n'):
        if line.startswith(key):
            return float(line.split('=')[-1])
    return None

def extract_product_size(primer_output, pair_index):
    key = f'PRIMER_PAIR_{pair_index}_PRODUCT_SIZE='
    for line in primer_output.split('\n'):
        if line.startswith(key):
            return int(line.split('=')[-1])
    return None

# ---------- Шаг 2: Проверка вторичных структур с помощью ViennaRNA ----------
def analyze_with_vienna(sequence):
    structure, mfe = RNA.fold(sequence)
    has_hairpin = '(' in structure and ')' in structure
    return {
        'structure': structure,
        'mfe': mfe,
        'has_hairpin': has_hairpin
    }

# ---------- Полный анализ ----------
def analyze_primer_for_gene(gene_name):
    print(f"Шаг 0: Загрузка последовательности для гена '{gene_name}'...")
    sequence = fetch_gene_sequence(gene_name)

    print('\nШаг 1: Генерация праймеров с помощью Primer3...')
    primers = run_primer3(sequence)

    print('\nШаг 2: Проверка праймеров с помощью ViennaRNA...')
    primer_issues = analyze_primers_with_criteria(primers)

    results = {
        'GeneSequence': sequence,
        'Primer3': primers,
        'PrimerIssues': primer_issues,
    }

    display_results(results)
    return results

# ---------- Переработанный вывод ----------
def format_primer_issues(primer_issues):
    formatted_issues = []
    for issue in primer_issues:
        pair = issue['Pair']
        problems = issue['Issues']
        if problems:
            formatted_issues.append(f"Пара {pair + 1}:\n  - " + '\n  - '.join(problems))
    return '\n\n'.join(formatted_issues)

def display_results(results):
    print('=== Итоговый анализ праймеров ===\n')
    print(f"Длина последовательности гена: {len(results['GeneSequence'])} bp\n")

    primer_issues = results['PrimerIssues']
    if primer_issues:
        print('Проблемы, найденные при анализе праймеров:\n')
        print(format_primer_issues(primer_issues))
    else:
        print('Все праймеры соответствуют критериям.')

In [22]:
gene_name = "MT-ND1"
results = analyze_primer_for_gene(gene_name)

Шаг 0: Загрузка последовательности для гена 'MT-ND1'...
Последовательность гена 'MT-ND1' успешно загружена.

Шаг 1: Генерация праймеров с помощью Primer3...

Шаг 2: Проверка праймеров с помощью ViennaRNA...
=== Итоговый анализ праймеров ===

Длина последовательности гена: 357798456 bp

Проблемы, найденные при анализе праймеров:

Пара 1:
  - Шпилька в левом праймере с ΔG = -2.0 ккал/моль
  - Шпилька в правом праймере с ΔG = -1.899999976158142 ккал/моль

Пара 2:
  - Шпилька в левом праймере с ΔG = -2.0 ккал/моль
  - Шпилька в правом праймере с ΔG = -1.899999976158142 ккал/моль

Пара 3:
  - Температура отжига левого праймера (62.625°C) вне допустимого диапазона
  - Шпилька в левом праймере с ΔG = -2.0 ккал/моль
  - Шпилька в правом праймере с ΔG = -1.899999976158142 ккал/моль

Пара 4:
  - Температура отжига правого праймера (62.016°C) вне допустимого диапазона
  - Шпилька в левом праймере с ΔG = -2.0 ккал/моль
  - Шпилька в правом праймере с ΔG = -1.899999976158142 ккал/моль

Пара 5:
  - 

# analysis using IDT API

In [14]:
# ---------- Функция для получения токена ----------
def get_access_token(client_id, client_secret, idt_username, idt_password):
    """
    Получает токен доступа для API OligoAnalyzer.
    """
    try:
        # Формирование HTTP-запроса
        authorization_string = b64encode(bytes(client_id + ":" + client_secret, "utf-8")).decode()
        request_headers = {
            "Content-Type": "application/x-www-form-urlencoded",
            "Authorization": "Basic " + authorization_string
        }
                    
        data_dict = {
            "grant_type": "password",
            "scope": "test",
            "username": idt_username,
            "password": idt_password
        }
        request_data = parse.urlencode(data_dict).encode()

        post_request = request.Request(
            "https://eu.idtdna.com/Identityserver/connect/token",
            data=request_data,
            headers=request_headers,
            method="POST"
        )

        # Отправка запроса и получение ответа
        response = request.urlopen(post_request)
        body = response.read().decode()

        if response.status != 200:
            raise RuntimeError("Ошибка получения токена: " + response.status + "\n" + body)

        body_dict = json.loads(body)
        return body_dict["access_token"]
    except Exception as e:
        print(f"Ошибка при получении токена: {e}")
        return None


# ---------- Функция анализа Self-Dimer ----------
def analyze_self_dimer(primer, api_token):
    """
    Анализ гомодимеров (Self-Dimer) для праймера с помощью OligoAnalyzer API.
    """
    url = f'https://eu.idtdna.com/restapi/v1/OligoAnalyzer/SelfDimer'
    headers = {
        'Authorization': f'Bearer {api_token}'
    }
    params = {
        'primary': primer
    }

    try:
        print(f"Проверка Self-dimer для праймера: {primer}")
        response = requests.post(url, headers=headers, params=params)
        response.raise_for_status()  # Проверяет наличие HTTP ошибок
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Ошибка при анализе Self-Dimer: {e}")
        return None


# ---------- Функция анализа Hetero-Dimer ----------
def analyze_hetero_dimer(forward_primer, reverse_primer, api_token):
    """
    Анализ гетеродимеров (Hetero-Dimer) для двух праймеров с помощью OligoAnalyzer API.
    """
    url = f'https://eu.idtdna.com/restapi/v1/OligoAnalyzer/HeteroDimer'
    headers = {
        'Authorization': f'Bearer {api_token}'
    }
    params = {
        'primary': forward_primer,
        'secondary': reverse_primer
    }

    try:
        print(f"Проверка Hetero-dimer для праймеров: {forward_primer} и {reverse_primer}")
        response = requests.post(url, headers=headers, params=params)
        response.raise_for_status()  # Проверяет наличие HTTP ошибок
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Ошибка при анализе Hetero-Dimer: {e}")
        return None


# ---------- Функция визуализации димеров ----------
def visualize_dimer(sequence, bonds, delta_g, base_pairs, 
                    start_position=0, top_padding=0, bond_padding=0, bottom_padding=0, 
                    title="Dimer Visualization", secondary_sequence=None):
    """
    Текстовая визуализация димера (гомодимер или гетеродимер).
    """
    # Если вторая цепь не указана, берем перевернутую (для гомодимеров)
    if secondary_sequence is None:
        secondary_sequence = sequence[::-1]

    # Формируем верхнюю строку с учетом отступа
    top_line = " " * (top_padding + start_position) + f"5'  {sequence}  3'"

    # Формируем строку связей с учетом сдвига
    connection_line = " " * (4 + bond_padding + start_position)
    for bond in bonds:
        if bond == 2:  # Полная комплементарность
            connection_line += "|"
        elif bond == 1:  # Дополнительная комплементарность
            connection_line += "."
        else:
            connection_line += " "

    # Формируем нижнюю строку с учетом отступа
    bottom_line = " " * (bottom_padding + start_position) + f"3'  {secondary_sequence}  5'"

    # Приводим строки к общей длине (выровнять линии)
    max_length = max(len(top_line), len(connection_line), len(bottom_line))
    top_line = top_line.ljust(max_length)
    connection_line = connection_line.ljust(max_length)
    bottom_line = bottom_line.ljust(max_length)

    # Вывод визуализации
    print(f"\n{title}\nDelta G: {delta_g} ккал/моль | Base Pairs: {base_pairs}")
    print(top_line)
    print(connection_line)
    print(bottom_line)
    print("-" * max_length)


In [16]:
# Пример работы с токеном
client_id = "21"          # Укажите ваш client ID
client_secret = "6d31fff8-d7cf-4ee4-98d8-d1ba69940f65" # Укажите ваш client secret
idt_username = "takul"   # Укажите ваш логин
idt_password = "ULAsFp5J6zjS24E"   # Укажите ваш пароль

api_token = get_access_token(client_id, client_secret, idt_username, idt_password)
if api_token:
    print(f"Токен успешно получен: {api_token}")
    
    forward_primer = "ACCTCCTCCTCTTCTGGTTGGT"
    reverse_primer = "TTCCGCTCCTGGATGTCCCTTG"
    
    # Анализ димеров
    forward_dimer = analyze_self_dimer(forward_primer, api_token)
    reverse_dimer = analyze_self_dimer(reverse_primer, api_token)
    hetero_dimer = analyze_hetero_dimer(forward_primer, reverse_primer, api_token)
    
    # Визуализация димеров
    if forward_dimer and isinstance(forward_dimer, list):
        for dimer_result in forward_dimer:
            visualize_dimer(
                forward_primer,
                dimer_result['Bonds'],
                dimer_result['DeltaG'],
                dimer_result['BasePairs'],
                dimer_result.get('StartPosition', 0),
                dimer_result.get('TopLinePadding', 0),
                dimer_result.get('BondLinePadding', 0),
                dimer_result.get('BottomLinePadding', 0),
                title="Forward Self-Dimer"
            )
    # ADD REVERSE PRIMER SELFDIMER
    if hetero_dimer and isinstance(hetero_dimer, list):
        for dimer_result in hetero_dimer:
            visualize_dimer(
                forward_primer,
                dimer_result['Bonds'],
                dimer_result['DeltaG'],
                dimer_result['BasePairs'],
                dimer_result.get('StartPosition', 0),
                dimer_result.get('TopLinePadding', 0),
                dimer_result.get('BondLinePadding', 0),
                dimer_result.get('BottomLinePadding', 0),
                title="Hetero-Dimer",
                secondary_sequence=reverse_primer
            )
else:
    print("Анализ не выполнен.")

Токен успешно получен: 0c622f18c000c5f056b6fe78b9169dae
Проверка Self-dimer для праймера: ACCTCCTCCTCTTCTGGTTGGT
Проверка Self-dimer для праймера: TTCCGCTCCTGGATGTCCCTTG
Проверка Hetero-dimer для праймеров: ACCTCCTCCTCTTCTGGTTGGT и TTCCGCTCCTGGATGTCCCTTG

Forward Self-Dimer
Delta G: -4.41 ккал/моль | Base Pairs: 3
    5'  ACCTCCTCCTCTTCTGGTTGGT  3'
        |||            ...        
3'  TGGTTGGTCTTCTCCTCCTCCA  5'    
----------------------------------

Forward Self-Dimer
Delta G: -4.41 ккал/моль | Base Pairs: 3
5'  ACCTCCTCCTCTTCTGGTTGGT  3'
    |||  .          .  ...    
3'  TGGTTGGTCTTCTCCTCCTCCA  5'
------------------------------

Forward Self-Dimer
Delta G: -3.07 ккал/моль | Base Pairs: 2
 5'  ACCTCCTCCTCTTCTGGTTGGT  3'
      .  ||         ..  .      
3'  TGGTTGGTCTTCTCCTCCTCCA  5' 
-------------------------------

Forward Self-Dimer
Delta G: -3.07 ккал/моль | Base Pairs: 2
5'  ACCTCCTCCTCTTCTGGTTGGT  3'  
        .  ||      ..  .        
  3'  TGGTTGGTCTTCTCCTCCTCCA  5'
----------